In [1]:
import pandas as pd 
import numpy as np 
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passi

In [2]:
d1 = pd.read_csv("./train.csv")
d1

,filename,category
0,45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg,3
1,f74d1a5fc2498bbbfa045c74e3cc333e.jpg,3
2,f6c172096818c5fab10ecae722840798.jpg,3
3,251ffd610399ac00fea7709c642676ee.jpg,3
4,73c7328b8eda399199fdedec6e4badaf.jpg,3
...,...,...
105387,047a60001de0331608ba64092cc7ae2b.jpg,25
105388,ea39ac66ccdc4b4d4c6443f6c54d8ae3.jpg,25
105389,6215f8c52c5bbcfe3e63e0f3ac6265f8.jpg,25
105390,1733d8286f6658149c7b7cdeb40d6461.jpg,25


In [3]:
import os
base_dir = '/Users/kritsatornsaengweang/Downloads/shopee-product-detection-dataset'
train_dir = os.path.join(base_dir, 'train/train')
validation_dir = os.path.join(base_dir, 'train/test')
train_00_dir = os.path.join(train_dir, '00')
validation_00_dir = os.path.join(validation_dir, '00')

In [4]:
train_00_fnames = os.listdir(train_00_dir)
validation_00_fnames = os.listdir(validation_00_dir)
print(train_00_fnames[:10])

['b2c6d0de8ce0ee3c398054ff6cf89a0a.jpg', '99e4583429490d513a7ca78fd83c9e87.jpg', '28123f6c0f8146e4b39ca9dbef607c6a.jpg', 'f28873a320004cc3c1affb451b218906.jpg', 'f6de339751b4df47da15a4273a9d971c.jpg', '0d52ed10de9e46ce480641a9a9fc2c67.jpg', 'b0f4011de5a9fa31606933287007f216.jpg', 'f1036536bd89d21d84c399e5d04e294e.jpg', 'a6de77dbf97a21353362643e6e57fc5f.jpg', '0eae6ce2d1e392d4d09fc8724cdcb272.jpg']


In [5]:
print('Total 00 : ', len(train_00_fnames))
print('Total Val 00 : ' ,len(validation_00_fnames))

Total 00 :  2583
Total Val 00 :  100


In [6]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 101192 images belonging to 42 classes.
Found 4200 images belonging to 42 classes.


# dropout

In [7]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-06-28 17:28:09--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 2404:6800:4001:806::2010, 172.217.31.80, 172.217.166.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2404:6800:4001:806::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  3.82MB/s    in 19s     

2020-06-28 17:28:28 (4.37 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [8]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
  layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 768)


In [9]:
from tensorflow.keras.optimizers import RMSprop


x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation='relu')(x)

x = layers.Dropout(0.2)(x)

x = layers.Dense(1, activation='sigmoid')(x)


model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])

In [1]:
from tensorflow.keras.optimizers import SGD

unfreeze = False

]
for layer in pre_trained_model.layers:
  if unfreeze:
    layer.trainable = True
  if layer.name == 'mixed6':
    unfreeze = True


model.compile(loss='binary_crossentropy',
              optimizer=SGD(
                  lr=0.00001, 
                  momentum=0.9),
              metrics=['acc'])

NameError: name 'pre_trained_model' is not defined